In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# importing some usefull librariies

import glob
from tqdm.auto import tqdm
import random
import pandas as pd
import numpy as np
import os
from itertools import chain
import glob
import shutil
import tensorflow as tf

from PIL import Image
import os.path
import imagehash
import multiprocessing


pd.set_option('display.max_colwidth', None)

In [ ]:
DATA_DIR = 'data'

In [ ]:
data = tf.keras.utils.image_dataset_from_directory(DATA_DIR)  # Import dataset from data_dir into a image dataset

In [ ]:
hashing_method = [imagehash.average_hash,imagehash.phash,imagehash.whash,imagehash.dhash] # Here defining a list of hashes we will use

def return_hash(self,row_idx): # here we assume that the column name which contains frame  name is titled 'frame_name'
    '''
    INPUT
    row_idx: int; row number of dataframe containing frame name
    
    OUTPUT
    hashes of image: strings
    '''
    Hash = [] # creating an empty list to store the hash of each image based upon all the hashing method defined above
    img_read = Image.open(os.path.join(img_real_dir,row_idx['frame_name']))
    for func_hash in hashing_method:
        Hash.append(str(func_hash(img_read))) 

    return Hash[0],Hash[1],Hash[2],Hash[3] # returning the hash of an image based upon different algorithms

# we have defined a function which will return the hashes of image and 




tqdm.pandas() # we will use multi processing to engage all cores of cpu
df_dum = df.copy() # keeping a copy of dataframe
rows_iter = (row for _, row in df_dum.iterrows()) # row list 
pool = multiprocessing.Pool() # initiating the multiprocessing pool

# Here we map the function defined above to all the rows(frames) inside dataframe and create four columns titles hash1, hash2, hash3, hash4
# corresponding to the hash algoithms defined above
df_dum['hash1'],df_dum['hash2'],df_dum['hash3'],df_dum['hash4'] = zip(*list(tqdm(pool.imap(return_hash,rows_iter),total=len(df_dum)))) 

pool.close()
pool.join()

# now we will create a set of hash1 column and remove the duplicate rows and then further apply it to remaning hashes

df_dum = df_dum.groupby(['hash1'], as_index=False)[['frame_name','hash2','hash3','hash4']].agg(lambda x: list(set(x)))

df_dum['hash2'] = df_dum['hash2'].apply(lambda x: x[0]) # this returns the remaining hashes in a list containg duplicate hashes hence we are 
# taking the first hash out of list for hash2

df_dum = df_dum.groupby(['hash2'], as_index=False)[['frame_name','hash3','hash4']].agg(lambda x: list(set(list(chain.from_iterable(x)))))
df_dum['hash3'] = df_dum['hash3'].apply(lambda x: x[0])

df_dum = df_dum.groupby(['hash3'], as_index=False)[['frame_name','hash4']].agg(lambda x: list(set(list(chain.from_iterable(x)))))
df_dum['hash4'] = df_dum['hash4'].apply(lambda x: x[0])

df_dum = df_dum.groupby(['hash4'], as_index=False)[['frame_name']].agg(lambda x: list(set(list(chain.from_iterable(x)))))

df_dum['frame_name'] = df_dum['frame_name'].apply(lambda x: x[0])
df_dum.drop(columns='hash4') # removing the last column


In [ ]:
df_dum # it is the final dataframe containg unique imgaes name